# Chapter 12 Parallel Programming with Cython

本章将介绍Cython的基于线程的并行化计算。我们将会集中在Cython的<font face='consolas'>prange</font>函数上，其将允许我们将一个串行的for循环改写成多线程并行版本。

在介绍<font face='consolas'>prange</font>之前，我们必须先理解Python运行时和本地线程的交互，其涉及了Cpython的global interpreter lock。

### I. Thread-Based Parallelism and the Global Interpreter Lock
由于CPyhton的内存管理不是线程安全的，所以引入GIL。CPython的GIL是一个互斥锁，防止多个本地线程同时执行Python字节码（bytecodes）。换句话说，在执行CPython程序的过程中，CPython的GIL始终只允许一个本地（或系统）线程执行Python字节码。GIL不仅影响Python-level代码，也影响Python/C API整体。
- GIL是管理Python对象内存所必须的
- 不操作Python对象的C代码可以不受GIL管束
- GIL只针对CPython，其他Python实现如Jython，IronPython和PyPy无需GIL。

由于Cython代码被编译而非解释，其运行的并非Python字节码，我们可以通过Cython绕过GIL，实现基于线程的并行计算。

在运行Cython并行代码之前，我们首先需要管理GIL。Cython提供了两种管理机制：[1]nogil函数属性，[2]with  nogil context manager

#### The nogil Function Attribute
我们可以为函数配置nogil属性，表明其允许在释放GIL的情况下使用。在配置nogil属性的函数中，我们不能创建Python对象或与Python对象交互（包括静态定义的Python对象）。


In [ ]:
cdef int kernel(double complex z, double z_max, int n_max) nogil:


我们也可以声明导入的C和C++函数为nogil。**page 221**


In [ ]:
cdef extern from "math.h":
    double sin(double x) nogil
    double cos(double x) nogil
    double tan(double x) nogil
    # ...
# 或者：
cdef extern from "math.h" nogil:
    double sin(double x)
    double cos(double x)
    double tan(double x)
    # ...

#### The with nogil Context Manager
<font face='consolas'>with nogil:</font> 下的代码在释放GIL的情况下运行。也可以在该context manager内定义子context manager <font face='consolas'>with gil:</font> ，临时获取GIL以raise an exception或执行设计Python object的其他操作。

执行基于线程的并行计算的最简单的途径是使用已经实现并行计算的外部库。另一种途径，也是本章重点，是使用<font face='consolas'>prange</font>

In [1]:
# ...declare and initialize C arguments...
with nogil: # run without the GIL in place
    result = kernel(z, z_max, n_max)
# GIL reacquired
print result

NameError: name 'nogil' is not defined

### II. Using prange to Parallelize Loops
julia set的Cython串行循环实现的运行时间是2.24s。

In [13]:
%%file julia.pyx
from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

import numpy as np

cdef inline double norm2(double complex z) nogil:
    return z.real * z.real + z.imag * z.imag


cdef int escape(double complex z,
                double complex c,
                double z_max,
                int n_max) nogil:

    cdef:
        int i = 0
        double z_max2 = z_max * z_max

    while norm2(z) < z_max2 and i < n_max:
        z = z * z + c
        i += 1

    return i


@boundscheck(False)
@wraparound(False)
def calc_julia(int resolution, double complex c,
               double bound=1.5, double z_max=4.0, int n_max=1000):

    cdef:
        double step = 2.0 * bound / resolution
        int i, j
        double complex z
        double real, imag
        int[:, ::1] counts

    counts = np.zeros((resolution+1, resolution+1), dtype=np.int32)

    for i in prange(resolution + 1, nogil=True,
                    schedule='static', chunksize=1):
        real = -bound + i * step
        for j in range(resolution + 1):
            imag = -bound + j * step
            z = real + imag * 1j
            counts[i,j] = escape(z, c, z_max, n_max)

    return np.asarray(counts)

@boundscheck(False)
@wraparound(False)
def julia_fraction(int[:,::1] counts, int maxval=1000):
    cdef:
        int total = 0
        int i, j, N, M
    N = counts.shape[0]; M = counts.shape[1]

    for i in prange(N, nogil=True):
        for j in range(M):
            if counts[i,j] == maxval:
                total += 1
    return total / float(counts.size)

Overwriting julia.pyx


In [14]:
%%file setup_julia.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize

ext_modules = [
    Extension(
        'julia',
        ['julia.pyx'],
        extra_compile_args=['/openmp'],
        extra_link_args=['/openmp'],
    )
]
setup(name='julia',ext_modules=cythonize(ext_modules))

Overwriting setup_julia.py


In [15]:
!python setup_julia.py build_ext --inplace --compiler=msvc

Compiling julia.pyx because it changed.
Cythonizing julia.pyx
running build_ext
building 'julia' extension
creating build
creating build\temp.win-amd64-2.7
creating build\temp.win-amd64-2.7\Release
C:\Users\Think X260\AppData\Local\Programs\Common\Microsoft\Visual C++ for Python\9.0\VC\Bin\amd64\cl.exe /c /nologo /Ox /MD /W3 /GS- /DNDEBUG "-ID:\Program Files\Anaconda\include" "-ID:\Program Files\Anaconda\PC" /Tcjulia.c /Fobuild\temp.win-amd64-2.7\Release\julia.obj /openmp
julia.c
julia.c(2092) : warning C4244: '=' : conversion from 'Py_ssize_t' to 'int', possible loss of data
julia.c(2093) : warning C4244: '=' : conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\Think X260\AppData\Local\Programs\Common\Microsoft\Visual C++ for Python\9.0\VC\Bin\amd64\link.exe /DLL /nologo /INCREMENTAL:NO "/LIBPATH:D:\Program Files\Anaconda\libs" "/LIBPATH:D:\Program Files\Anaconda\PCbuild\amd64" /EXPORT:initjulia build\temp.win-amd64-2.7\Release\julia.obj "/OUT:D:\Program Files\Anaco

In [5]:
import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x3e07ac8>)

In [16]:
import julia
jl = julia.calc_julia(1000,(0.322 + 0.05j))
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(np.log(jl))

%timeit julia.calc_julia(1000,(0.322 + 0.05j))

ImportError: DLL load failed: 应用程序无法启动，因为应用程序的并行配置不正确。有关详细信息，请参阅应用程序事件日志，或使用命令行 sxstrace.exe 工具。